# Importing the required libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import random
import math

# Defining a function to execute the strategy

In [2]:
def make_trade(df, window_size, z_score_threshold, stop_loss, position_size, portfolio_value, position, PnL):
    trades = []
    daily_returns = []
    z_score_list = []
    stop_loss_level = 0
    for i in range(window_size, len(df)):
        window_data = df["Close Price"][i-window_size:i]
        mean = window_data.mean()
        std = window_data.std()
        z_score = (df["Close Price"][i] - mean) / std
        z_score_list.append(z_score)
        if abs(z_score) > z_score_threshold:
            if z_score > 0:
                trade_size = int((portfolio_value * position_size) / df["Open Price"][i])
                position += trade_size
                PnL -= trade_size * df["Open Price"][i]
                trades.append((df["Date"][i], "Buy", trade_size, df["Open Price"][i]))
                stop_loss_level = df["Open Price"][i] * (1 - stop_loss)
            else:
                trade_size = int((portfolio_value * position_size) / df["Open Price"][i])
                position -= trade_size
                PnL += trade_size * df["Open Price"][i]
                trades.append((df["Date"][i], "Sell", trade_size, df["Open Price"][i]))
                stop_loss_level = df["Open Price"][i] * (1 + stop_loss)
        elif position > 0 and df["Low Price"][i] < stop_loss_level:
            PnL += position * stop_loss_level
            trades.append((df["Date"][i], "Stop Loss", position, stop_loss_level))
            position = 0
        else:
            pass

        PnL += position * (df["Close Price"][i] - df["Open Price"][i])
        daily_return = df["Close Price"][i] / df["Close Price"][i-1] - 1
        daily_returns.append(daily_return)
        
    ROI = PnL / portfolio_value
    max_drawdown = 1 - df["Close Price"].max() / df["Close Price"].cummax().max()
    daily_returns = np.array(daily_returns)
    Sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std()
    zsa = z_score_list

    return PnL, ROI, max_drawdown, daily_returns, (-(Sharpe_ratio)), zsa

# User I/P

In [3]:
window_size = 10
z_score_threshold = 2
stop_loss = 0.20
position_size = 0.30
position = 0
PnL = 0

In [4]:
portfolio_value = float(input("Enter your current portfolio value: "))

Enter your current portfolio value: 100000


# Testing the code on Equity data

In [5]:
df1 = pd.read_excel(r"/Users/raghavsrivastava/Desktop/Equity Data/IDFC LTD (HISTORICAL DATA).xlsx")

In [6]:
df2 = pd.read_excel(r"/Users/raghavsrivastava/Desktop/Equity Data/IDFCFIRST BANK (HISTORICAL DATA).xlsx")

In [7]:
df3 = pd.read_excel(r"/Users/raghavsrivastava/Desktop/Equity Data/IRFC HISTORICAL DATA.xlsx")

# O/P

In [8]:
PnL_1, ROI_1, max_drawdown_1, daily_returns_1, Sharpe_ratio_1, zsa_1 = make_trade(df1, window_size, z_score_threshold, stop_loss, position_size, portfolio_value, position, PnL)

In [9]:
PnL_2, ROI_2, max_drawdown_2, daily_returns_2, Sharpe_ratio_2, zsa_2 = make_trade(df2, window_size, z_score_threshold, stop_loss, position_size, portfolio_value, position, PnL)

In [10]:
PnL_3, ROI_3, max_drawdown_3, daily_returns_3, Sharpe_ratio_3, zsa_3 = make_trade(df3, window_size, z_score_threshold, stop_loss, position_size, portfolio_value, position, PnL)

In [11]:
print("IDFC LTD")

print("Final P&L INR: {:,.2f} ".format(PnL_1))
print("ROI : {:.2f}%".format(ROI_1 * 100))
print("Sharpe ratio : {:.2f} ".format(Sharpe_ratio_1))

IDFC LTD
Final P&L INR: 1,287,869.98 
ROI : 1287.87%
Sharpe ratio : -0.33 


In [12]:
print("IDFC FIRST BANK")
print("Final P&L INR : {:,.2f} ".format(PnL_2))
print("ROI : {:.2f}%".format(ROI_2 * 100))
print("Sharpe ratio : {:.2f} ".format(Sharpe_ratio_2))

IDFC FIRST BANK
Final P&L INR : 4,125,799.60 
ROI : 4125.80%
Sharpe ratio : -0.18 


In [13]:
print("IRFC")
print("Final P&L INR : {:,.2f} ".format(PnL_3))
print("ROI : {:.2f}%".format(ROI_3 * 100))
print("Sharpe ratio : {:.2f} ".format(Sharpe_ratio_3))

IRFC
Final P&L INR : 292,212.58 
ROI : 292.21%
Sharpe ratio : -0.34 
